## Часть 1
Напишите скрипт, который скачивает все данные с выборов губернатора СПб 2019г для всех избирательных участков. 

Входная точка [по ссылке](http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import numpy as np

In [2]:
url = "http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222"

r = requests.get(url)
html = r.text
soup = BeautifulSoup(html, "lxml")
refs = soup.find_all("a")

In [3]:
uik_refs = []
names = []
for ref in refs:
    if "Территориальная избирательная комиссия" in ref.text: 
        uik_refs.append(urljoin(url, ref.get("href")))
        names.append(ref.text)
        
refs_dict = dict(zip(names, uik_refs))

In [4]:
table = soup.find_all("table")[-2]
rows = table.find_all("tr")
data = []
for row in rows:
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

data.remove(data[12])
idxs = [d[1] for d in data[1:]]

In [5]:
refs_names = []
all_data = []
df = pd.DataFrame()
for name, ref in refs_dict.items():
    req = requests.get(ref)
    soup_ref = BeautifulSoup(req.text, "lxml")
    table = soup_ref.find_all("table")[-1]
    rows = table.find_all("tr")
    data_ref = []
    for row in rows[0:1]:
        cols = row.find_all("td")
        cols = [ele.text.strip() for ele in cols]
        col_names = [ele for ele in cols if ele]
    for row in rows[1:]:
        cols = row.find_all("td")
        cols = [ele.text.strip() for ele in cols]
        data_ref.append(np.array([int(ele.split("\n")[0]) for ele in cols if ele]))
    data_ref.remove(data_ref[11])
    if not df.empty:
        df_2 = pd.DataFrame(data_ref, columns=pd.MultiIndex.from_tuples(zip([name]*len(col_names), col_names)), index=idxs)
        df = pd.concat([df, df_2], axis=1)
    else:
        df = pd.DataFrame(data_ref, columns=pd.MultiIndex.from_tuples(zip([name]*len(col_names), col_names)), index=idxs)


<ipython-input-5-2dc755f7323d>:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data_ref.remove(data_ref[11])


In [6]:
df.T.to_csv("elections.csv")

In [7]:
df.T

Число избирателей, внесенных в список избирателей на момент окончания голосования  \
Территориальная избирательная комиссия №1  УИК №1                                                  1803                                   
                                           УИК №2                                                  1466                                   
                                           УИК №3                                                  2092                                   
                                           УИК №4                                                  1056                                   
                                           УИК №5                                                  1827                                   
...                                                                                                 ...                                   
Территориальная избирательная комиссия №30 УИК №2350                                                284                                   
                                           УИК №2351                                                132                                   
                                           УИК №2352                                                320                                   
                                           УИК №2355                                                485                                   
                                           УИК №2356                                                620                                   

                                                      Число избирательных бюллетеней, полученных участковой избирательной комиссией  \
Территориальная избирательная комиссия №1  УИК №1                                                  1200                               
                                           УИК №2                                                  1100                               
                                           УИК №3                                                  1600                               
                                           УИК №4                                                  1000                               
                                           УИК №5                                                  1400                               
...                                                                                                 ...                               
Территориальная избирательная комиссия №30 УИК №2350                                                300                               
                                           УИК №2351                                                150                               
                                           УИК №2352                                                350                               
                                           УИК №2355                                                500                               
                                           УИК №2356                                                650                               

                                                      Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования  \
Территориальная избирательная комиссия №1  УИК №1                                                   587                                                     
                                           УИК №2                                                   433                                                     
                                           УИК №3                                                   576                                                     
                                           УИК №4                                                   318                                 